# старый

In [ ]:
import telebot
from telebot import types
from random import *
import requests

bot = telebot.TeleBot('5106587413:AAF72ed2pSaHYWuNymSudE_hHEZuxtygMWw')

USERS = dict()

# #
# Клава с функциями
def keyboard_menu(message, text):
    keyboard = types.InlineKeyboardMarkup(row_width=3)

    but1 = types.InlineKeyboardButton(text='Изменить уровень английского', callback_data='changе_level')
    but2 = types.InlineKeyboardButton(text='Потренировать грамматику', callback_data='gram')
    but3 = types.InlineKeyboardButton(text='Потренировать слова', callback_data='training_words')
    but4 = types.InlineKeyboardButton(text='Тест на слова', callback_data='test_words')
    but5 = types.InlineKeyboardButton(text='Yandex Translate', callback_data='translate')

    keyboard.add(but1, but2, but3, but4, but5)
    bot.send_message(chat_id = message.from_user.id, text = text, parse_mode = "HTML", reply_markup = keyboard)


# Клава с выбором уровня
def keyboard_level(message, text):
    keyboard = types.InlineKeyboardMarkup(row_width=1)

    but1 = types.InlineKeyboardButton(text='Elementary', callback_data='eleme')
    but2 = types.InlineKeyboardButton(text='Pre-Intermediate', callback_data='prein')
    but3 = types.InlineKeyboardButton(text='Intermediate', callback_data='inter')
    but4 = types.InlineKeyboardButton(text='Upper-Intermediate', callback_data='upper')

    keyboard.add(but1, but2, but3, but4)
    bot.send_message(chat_id=message.from_user.id, text = text, parse_mode = "HTML", reply_markup = keyboard)


# Старт, выкидывает клаву с выбором уроня или гланое меню
@bot.message_handler(content_types=['start'])
def start_message(message):

    if str(message.chat.id) in USERS:
        keyboard_menu(message, "<b>Чем бы вы хотели заняться?</b> ")
    else:
        keyboard_level(message, "(какое нибудь приветствие) <b>Выберите уровень английского</b> ")


# Ловит выбор уровня, или нажатие на клавишу "главное меню"
# Выкидывает клаву с главным меню и создаёт пользователя
@bot.callback_query_handler(func=lambda c: c.data in ['eleme', 'prein', 'inter', 'upper', 'mainmenu'])
def main_menu(callback):

    if callback.data != 'mainmenu':
        USERS.update( { str(callback.message.chat.id): {'level': callback.data, 'mod': 'None'} } )

    keyboard_menu(callback, "<b>Чем бы вы хотели заняться?</b> ")


# Ловит нажатие на кнопку "изменить уровень"
# выкидывает клаву с уровнями и удаляет пользователя
@bot.callback_query_handler(func=lambda c: c.data == 'changе_level')
def level_english(message):
    USERS.pop(str(message.message.chat.id))

    keyboard_level(message, "<b>Выберите новый уровень</b> ")



@bot.callback_query_handler(func=lambda c: c.data == 'training_words')
def tr_w(callback):
    keyboard = types.InlineKeyboardMarkup(row_width=1)
    but1 = types.InlineKeyboardButton(text='С Англ на Русс', callback_data='trainingWords_en-ru')
    but2 = types.InlineKeyboardButton(text='С Русс на Англ', callback_data='trainingWords_ru-en')
    but3 = types.InlineKeyboardButton(text='Рандомно', callback_data='trainingWords_rndom')
    but4 = types.InlineKeyboardButton(text='Назад', callback_data='mainmenu')
    keyboard.add(but1, but2, but3, but4)

    bot.send_message(chat_id=callback.message.chat.id, text="Выберете режим", parse_mode="HTML", reply_markup=keyboard)


@bot.callback_query_handler(func=lambda c: c.data == 'test_words')
def tr_w(callback):
    keyboard = types.InlineKeyboardMarkup(row_width=1)
    but1 = types.InlineKeyboardButton(text='С Англ на Русс', callback_data='testWords_en-ru')
    but2 = types.InlineKeyboardButton(text='С Русс на Англ', callback_data='testWords_ru-en')
    but3 = types.InlineKeyboardButton(text='Рандомно', callback_data='testWords_rndom')
    but4 = types.InlineKeyboardButton(text='Назад', callback_data='mainmenu')
    keyboard.add(but1, but2, but3, but4)

    bot.send_message(chat_id=callback.message.chat.id, text="Выберете режим", parse_mode="HTML", reply_markup=keyboard)



@bot.callback_query_handler(func=lambda c: c.data == 'translate')
def trans(message):
    USERS[str(message.message.chat.id)]['mod'] = 'translate'

    bot.send_message(chat_id=message.from_user.id, text='Вы вводите текст, я вам перевод, для выхода в меню /menu',
                     parse_mode="HTML")


@bot.callback_query_handler(func=lambda c: c.data in ['trainingWords_en-ru','trainingWords_ru-en','trainingWords_rndom'])
def tr(callback):
    USERS[str(callback.message.chat.id)]['mod'] = '%%%'+USERS[str(callback.message.chat.id)]['level'] + callback.data[13:]
    bot.send_message(chat_id=callback.message.chat.id,
                     text='Я пишу вам слово, вы мне его перевод, и я вас проверяю, если ответ верный, '
                          'я просто пришлю следующее слово, для выхода напишите /menu',
                     parse_mode="HTML")

    bot.send_message(chat_id=callback.message.chat.id,
                     text='Начнём',
                     parse_mode="HTML")
    w1,w2 = choice(list(WORDS[USERS[str(callback.message.chat.id)]['mod'][3:14]].items()))
    USERS[str(callback.message.chat.id)]['mod'] = USERS[str(callback.message.chat.id)]['mod'] + w2
    bot.send_message(chat_id=callback.message.chat.id,
                     text=w1,
                     parse_mode="HTML")


# Тест слова 

test_mode = ""
answer_word=""
@bot.callback_query_handler(func=lambda c: c.data in ['testWords_en-ru','testWords_ru-en','testWords_random'])
def words_test(callback):
    if USERS[str(callback.message.chat.id)]['level'] == "eleme":
        lvl = "elem"
    else:
        lvl = USERS[str(callback.message.chat.id)]['level']

    w1_word = choice(list(WORDS[USERS[str(callback.message.chat.id)]['level'] + callback.data[9:]].keys()))
    w2_word = choice(list(WORDS[USERS[str(callback.message.chat.id)]['level'] + callback.data[9:]].keys()))
    w3_word = choice(list(WORDS[USERS[str(callback.message.chat.id)]['level'] + callback.data[9:]].keys()))
    global answer_word
    true_word, answer_word = choice(list(WORDS[USERS[str(callback.message.chat.id)]['level'] + callback.data[9:]].items()))


    words_list = []
    while True:
        n = randint(0,3)
        if n == 0:
            if true_word not in words_list:
                words_list.append(true_word)
        elif n == 1:
            if w1_word not in words_list:
                words_list.append(w1_word)
        elif n == 2:
            if w2_word not in words_list:
                words_list.append(w2_word)
        elif n == 3:
            if w3_word not in words_list:
                words_list.append(w3_word)
        if len(words_list) == 4:
            break

    def check(word):
        if word == true_word:
            return "прав"
        if word != true_word:
            return "неправ"


    KB_words = types.InlineKeyboardMarkup(row_width=1)
    but1 = types.InlineKeyboardButton(text=words_list[0], callback_data=check(words_list[0]))
    but2 = types.InlineKeyboardButton(text=words_list[1], callback_data=check(words_list[1]))
    but3 = types.InlineKeyboardButton(text=words_list[2], callback_data=check(words_list[2]))
    but4 = types.InlineKeyboardButton(text=words_list[3], callback_data=check(words_list[3]))
    KB_words.add(but1, but2, but3,but4)
    bot.edit_message_text(chat_id=callback.message.chat.id, message_id=callback.message.id, text=f"<b>Переведите слово:</b> {answer_word}", parse_mode="HTML", reply_markup=KB_words)


@bot.callback_query_handler(func=lambda n: n.data == "прав" or n.data == "неправ")
def check_word_answer(callback):
    KB_words_check = types.InlineKeyboardMarkup(row_width=1)
    but1 = types.InlineKeyboardButton(text="Еще слово", callback_data=test_mode)
    but2 = types.InlineKeyboardButton(text="Главное меню", callback_data=USERS[str(callback.message.chat.id)]['level'])
    KB_words_check.add(but1, but2)

    if callback.data == "прав":
        bot.edit_message_text(chat_id=callback.message.chat.id, message_id=callback.message.id, text="<b>Правильно!</b>", parse_mode="HTML", reply_markup=KB_words_check)
    if callback.data == "неправ":
        bot.edit_message_text(chat_id=callback.message.chat.id, message_id=callback.message.id, text=f"<b>Неправильно(( Правильный ответ:</b> {answer_word}", parse_mode="HTML",reply_markup=KB_words_check)



# Ловит и обрабатывает все текстовые сообщения от пользователя
@bot.message_handler(func=lambda c: True)
def read_message(message):
    mod = 'None'
    if str(message.chat.id) in USERS:
        mod = USERS[str(message.chat.id)]['mod']

    if message.text == '/start':
        start_message(message)

    if message.text == '/menu':
        keyboard_menu(message, "<b>Чем бы вы хотели заняться?</b> ")

    elif mod == 'translate':   # Если пользователь будучи в режиме перевода отправляет сообщение, бот отправляет перевод

        transl = translate(message.text, 'ru')

        bot.send_message(chat_id=message.from_user.id, text=transl, parse_mode="HTML")

    elif mod[:3] == '%%%':          # Если пользователь будучи в режиме тренировки слов отправляет сообщение
        print(USERS)
        if USERS[str(message.chat.id)]['mod'][14:] == message.text:
            USERS[str(message.from_user.id)]['mod'] = USERS[str(message.from_user.id)]['mod'][:14]
            w1, w2 = choice(list(WORDS[USERS[str(message.from_user.id)]['mod'][3:]].items()))
            USERS[str(message.from_user.id)]['mod'] = USERS[str(message.from_user.id)]['mod'] + w2
            bot.send_message(chat_id=message.from_user.id,
                             text=w1,
                             parse_mode="HTML")

bot.polling()

{}
